# AI Assisted Annotation 

Manual annotation is slow, tedious and costly. Faster labeling of 3D volumes using AI annotation models accelerates this process. Clara Train offers an AIAA API that easily integrates into common medical imaging viewers
<br><img src="screenShots/AIAASpeedup.png" alt="Drawing" style="height: 400px;"/><br>

AIAA is based on a server client model as shown below 
<br><img src="screenShots/AIAAClientServer.png" alt="Drawing" style="height: 400px;"/><br>



By the end of this notebook you will be able to:
- Start AIAA server 
- Load a deep grow model
- Annotate using deep grow 
- Load your model and use it for annotations  
- Stop AIAA server


## Prerequisites
- Nvidia GPU with 8Gb of memory   


### Resources
It might be helpful to watch the GTC Digital 2020 talk on Clara Train SDK 
- [S22563](https://developer.nvidia.com/gtc/2020/video/S22563)
Clara train Getting started: cover basics, BYOC, AIAA, AutoML 


## Lets get started
Before we get started let us check that we have an NVIDIA GPU available in the docker by running the cell below

In [ ]:
# following command should show all gpus available 
!nvidia-smi


## 1- Start AIAA server
The cell below will start the AIAA server


In [ ]:
AIAA_ROOT="/claraDevDay/AIAA/workspace/"
AIAA_PORT="5000"
!mkdir -p $AIAA_ROOT
!chmod 777 $AIAA_ROOT
print ("AIAA_ROOT is set to ",AIAA_ROOT)

In [ ]:
# To run from a terminal you should just run
# start_aas.sh --workspace <workspace> &  
import subprocess
a = subprocess.Popen(["start_aiaa.sh","--workspace",AIAA_ROOT])
!sleep 10 #AIAA need a bit of time to start 

Let us now check that the server has started by:
- Main url [http://localhost:5000/](http://clarasa-station:5000/) <br>
- APIS to list, upload, delete models [http://localhost:5000/docs/](http://clarasa-station:5000/docs/)
- Check logs [http://localhost:5000/logs](http://clarasa-station:5000/logs)
- List available models [http://localhost:5000/v1/models/](http://clarasa-station:5000/v1/models) 
- Dashboard [http://localhost:5000/dashboard/](http://clarasa-station:5000/dashboard) 

You can also use web APIs as curl command to do 
The cell below will get the last 15 lines of the logs.

In [ ]:
http_str="http://127.0.0.1:"+AIAA_PORT+"/logs/?lines=15"
!curl -X GET $http_str -H "accept: application/json" 


##  2- Load models in AIAA Server. 
For this notebook we can download models from [NGC](). Models on NGC are either:
- Annotation Models <br>
[Deep Extreme Cut: From Extreme Points to Object Segmentation](https://arxiv.org/abs/1711.09081)
<br><img src="screenShots/AIAAAnnotation.png" alt="Drawing" style="height: 400px;"/><br>

- Segmentation Models
- DeepGrow MMAR 
This is an interactive model to get you started with annotation. CNN takes in single channel (image) + use single click 
for foreground or background location then produces the segmentation. it is based on [Interactive segmentation of medical images through
fully convolution neural networks](https://arxiv.org/pdf/1903.08205.pdf)
<br><img src="screenShots/AIAADeepGrow.png" alt="Drawing" style="height: 400px;"/><br>



#### 2.1 Using NGC CLI to download model from NGC (Will NOT work for EA) 
You can see a [list of available pre-trained models](https://ngc.nvidia.com/containers/nvidia:clara-train-sdk) on NGC. 
You can also use `ngc registry model list nvidia/med/clara_*` to get a list of models.

In [ ]:
!ngc registry model list nvidia/clara_*
!ngc registry model list nvidia/med/clara_*


The cell below will download the the deep grow model from NGC
***Will Not work for EA***

In [ ]:
# model name from NGC is clara_train_deepgrow_aiaa_inference_only
http_str="http://127.0.0.1:"+AIAA_PORT+"/admin/model/clara_deepgrow"
!curl -X PUT $http_str \
     -H "accept: application/json" \
     -H "Content-Type: application/json" \
     -d '{"path":"nvidia/med/clara_train_deepgrow_aiaa_inference_only","version":"1"}'


The following cell will download the spleen model and load it into the AIAA server

In [ ]:
http_str="http://127.0.0.1:"+AIAA_PORT+"/admin/model/clara_ct_seg_spleen_no_amp"
!curl -X PUT $http_str \
     -H "accept: application/json" \
     -H "Content-Type: application/json" \
     -d '{"path":"nvidia/med/clara_ct_seg_spleen_no_amp","version":"1"}'

#### 2.2 Manually Download model from NGC 
If you have a private registry or have early access (EA) to nvidia's developer program, 
you can manually downloaded the MMAR form NGC by clicking the ... button as shown below
<br><img src="screenShots/DownloadFromNGC.png" alt="Drawing" style="height: 400px;"/><br>

**You should download zip file to /claraDevDay/AIAA/DownloadsFromNGC/**
 

Zip file would have the MMAR structure. 
Cell below would unzip the mmar then upload it to AIAA server 

In [ ]:
downloadedFile="Deepgrow2DV4EA.zip"

downloadDir=AIAA_ROOT+"../DownloadsFromNGC/"
modelName=downloadedFile[:-4]
%cd $downloadDir
!unzip $downloadedFile -d $modelName

In [ ]:
http_str="http://127.0.0.1:"+AIAA_PORT+"/admin/model/"+modelName
conf_str="config=@"+downloadDir+modelName+"/config/config_aiaa.json;type=application/json"
dataArg="data=@"+downloadDir+modelName+"/models/model.ts"
cmd='curl -X PUT "'+http_str +'" -F "'+conf_str +'" -F "'+dataArg +'"'
print(cmd)
!$cmd

#### 2.3 Check for models loaded into AIAA
Let us check the server and check that the model is uploaded. after running the cell below you should see the deep grow model name returned with the description 

In [ ]:
http_str="http://127.0.0.1:"+AIAA_PORT+"/v1/models"
!curl -X GET $http_str -H "accept: application/json"

## 3- AIAA Clients 

AIAA server can connect to any client that implements the APIs found [here](https://github.com/NVIDIA/ai-assisted-annotation-client
). 
NVIDIA has already implemented these APIs for a number of open source viewers as:


### 3.1. [3D Slicer](https://www.slicer.org/)

In order to use slicer you should: 
1. Install and setup slicer 3d following steps [here](https://github.com/NVIDIA/ai-assisted-annotation-client/tree/master/slicer-plugin)
    1. Download and install recent 3D Slicer Preview Release (4.11.x) from [here](http://download.slicer.org/).
    For Early access please use slicer 4.13 (unstable release) this is needed to enable 3d geepgrow models. 
    2. Start 3D Slicer and open the Extension manager
    3. Install NvidiaAIAssistedAnnotation extension (in Segmentation category), wait for the installation to complete, and click Restart
2. Install by searching for nvidia in the plugin manager
3. Configure plugin. Add the AIAA server location and make sure the session is enabled
   <br><img src="screenShots/SlicerConfig.png" alt="Drawing" style="height: 400px;"/>
4. You should load a volume and start trying the spleen and deep grow model as shown below
     
<br><img src="screenShots/Slicer.png" alt="Drawing" style="height: 400px;"/><br>

### 3.2. [MITK](https://www.mitk.org/wiki/The_Medical_Imaging_Interaction_Toolkit_(MITK))
MITK is another viewer that you can use with AIAA. 
You can download and install it [here](http://mitk.org/wiki/Downloads). 
Please make sure you install the release with nvidia AIAA.<br>
_Note_: Deep grow is not enabled yet in MITK 


### 3.3. [OHIF](https://docs.ohif.org/)
We have integrated AIAA with OHIF as a plugin, This has been adapted in to XNAT. 
Plugin code can be found as a branch of OHIF github [here](https://github.com/SachidanandAlle/Viewers) 


## 4- Delete Models from AIAA server  

In [ ]:
# if you need to change the model name 
#modelName="Deepgrow2DV4EA" 
http_str='"http://127.0.0.1:'+AIAA_PORT+'/admin/model/'+modelName+'"'
!curl -X DELETE $http_str


## 5- Stop AIAA server  

In [ ]:
!stop_aiaa.sh